# Modeling Internal Representation

A useful abstraction is thinking of each of the opponent's keywords as a random variable. We know the opponent has a keyword card with a certain number of fixed keywords, but not what the keywords are. However, we know they must be *a* word. So we may as well think of each of their words as having some probability of being each word possible, and refine the probability values based on context and revealed information.

Somewhat less obvious is that we may do the same for our own keywords; for each word, the distribution would simply have probability 1 for the keyword, and 0 for every other word.

# Guessing with Backtracking

To make a good guess, the objective would be to maximize the probability of our guess; this gives the highest likelihood of being correct and raises our expected score through interceptions.

Say we have keywords ["GARDEN", "MUSKET", "BOWL", "BAT"] and clues ("wing", "leaf", "powder"). We might notice right away that "wing" makes a lot of sense with "BAT", so we might choose the first code number to be index 3 and remove both from consideration. This simplifies our consideration to ["GARDEN", "MUSKET", "BOWL"] and clues ("leaf", "powder"), where we might identify more easily now that "leaf" corresponds to "GARDEN". Again, we might choose 0 as our next code number and remove both from consideration. This yields ["MUSKET", "BOWL"] and clues ("powder"). It is much easier to identify that "powder" probably corresponds to "MUSKET". We notice that guessing has somewhat of a recursive property. 

Let's think about how this might be represented mathematically. We might say that each keyword has a random variable associated with it

> K_0, K_1, K_2, K_3

We might then say the each code number has a random variable associated with it
> C_0, C_1, C_2

and each corresponding clue random variables (M is for "message").

> M_0, M_1, M_2 

Then we wish to maximize

> P(M_0="wing", M_1="leaf", M_2="powder").

To do so, we take the maximum of

> P(C_0=i) * P(M_0="wing", M_1="leaf", M_2="powder" | C_0=i) from i = 0 to 3.

This yields

> P(C_0=i) * P(M_0="wing", M_1="leaf", M_2="powder", C_0=i) / P(C_0=i)

The P(C_0=i) terms cancel! To continue, we find

> P(M_0="wing", M_1="leaf", M_2="powder", C_0=i)

> =

> P(M_1="leaf", M_2="powder" | M_0="wing", C_0=i) * P(M_0="wing",  C_0=i)

This is where we recall the example; when we thought about the remaining words, we didn't consider the fact that we had assigned one of the previous clues a code number. That was the utility of the reduction, that it made it more simple to think about; it follows then that the probability of the remaining M_1 and M_2 is independent of the choice of M_0 and C_0, and we may remove that condition.

> P(M_1="leaf", M_2="powder") * P(M_0="wing",  C_0=i) 

We see a recurrence! In its entirety we write it as 

> f(P(M_0, M_1, M_2)) = max(f(P(M_1, M_2=)) * P(M_0,  C_0=i)) from i = 0 to 3

We may calculate

> P(M_0,  C_0=i)

with a metric of our choice and our random variable representations! 

For clarity I should note that although I went over probabilities here, that only makes sense for a single keyword, which is what the Guesser deals with. To make this more general, when we deal with each keyword as a random variable, we should think about the *expected* probability.

We might be able to optimize with Dynamic Programming but let's worry about that later if it proves necessary.
I am going to use log probabilities becasue some of the probabilities we are working with might get very small, and it will help preserve precision. Additionally, adding and subtracting might yield us some speed benefits. This should also help in the future should we explore this more in the information theory context.



In [81]:
from typing import Sequence
import math

class RandomVariableTracker:
    def __init__(self, random_vars: Sequence[dict]):
        self.random_vars = random_vars
        
    def expected_log_probability_given_clue(self, random_var: dict, clue: str):
        # this would need to be more refined for the general case
        return random_var.get(clue, -math.inf) # P(X_i = clue)
        
    def max_expected_log_probability_guess(self, clues: tuple[str], var_indices=None):
        if not clues:
            return 0.0, tuple()
        
        var_indices = var_indices if var_indices is not None else tuple(range(len(self.random_vars))) 
        clue, *remaining_clues = clues
        remaining_clues = tuple(remaining_clues)
                
        max_log2_probability = -math.inf
        best_guess = None
        for i, var_index in enumerate(var_indices):
            remaining_var_indices = var_indices[:i] + var_indices[i + 1:]
            
            max_subproblem_log2_probability, guess = self.max_expected_log_probability_guess(remaining_clues, remaining_var_indices)
            log2_probability = max_subproblem_log2_probability + self.expected_log_probability_given_clue(self.random_vars[var_index], clue)
            
            if log2_probability > max_log2_probability:
                max_log2_probability = log2_probability
                best_guess = (var_index,) + guess
                
        return max_log2_probability, best_guess

In [82]:
import decryptogame as dg
from itertools import permutations

# let's see if our recurrence yields a good guessing heuristic!

# generate random keywords
keywords = next(dg.generators.RandomKeywordCards())[0]
print(keywords)

# initialize tracker
random_vars = [{keyword: 0.0} for keyword in keywords] # log(1.0) == 0.0

tracker = RandomVariableTracker(random_vars)

# try every code
codes = list(permutations(range(len(keywords)), 3))
clues = [tuple(keywords[code_num] for code_num in code) for code in codes]
print(clues[:5])

log2_probabilities_and_guesses = [tracker.max_expected_log_probability_guess(clue) for clue in clues]

probabilities = [math.exp(log_probability) for log_probability, _ in log2_probabilities_and_guesses]
guesses = [guess for _, guess in log2_probabilities_and_guesses]

# check that probabilities made sense and that each guess was correct
print(set(probabilities)) # should all be 1
print(guesses == codes) # should be True

('OGRE', 'FIRE', 'PERFUME', 'DOG')
[('OGRE', 'FIRE', 'PERFUME'), ('OGRE', 'FIRE', 'DOG'), ('OGRE', 'PERFUME', 'FIRE'), ('OGRE', 'PERFUME', 'DOG'), ('OGRE', 'DOG', 'FIRE')]
{1.0}
True


Nice! To make an effective Guesser beyond this prototype, I wan't to reuse the max_expected_log_probability_guess function, and try several different expected_log_probability_given_clue methods. The clue_probability will also need to be a bit more complex. Rather than start a messy inheritance hierarchy, I think it would make more sense to inject these as strategies, particularly the expected_log_probability_given_clue function. Let's refactor to make this more flexible for testing.

In [83]:
from functools import partial, reduce

# abstract/encapsulate RadnomVariable so don't have to think about log logic as much

def softmax_combine(log_x, log_y):
    return log_x + math.log1p(math.exp(log_y - log_x))

class RandomVariable:
    def __init__(self, log_probabilities):
        self.log_probabilities = log_probabilities

    def log_expectation(self, key_to_log_val_func): # inject function, this is like log(E[f(X)])
        if not self.log_probabilities:
            return -math.inf
        
        return reduce(softmax_combine,
                      (log2_prob + key_to_log_val_func(key) for key, log2_prob in self.log_probabilities.items())
                      )

     
def max_expected_log_probability_guess(clue_and_keyword_to_log_prob_func, random_vars: Sequence[RandomVariable], clues: tuple[str]) -> [float, tuple[int]]:
    keyword_to_log_prob_given_clue = [partial(clue_and_keyword_to_log_prob_func, clue) for clue in clues]

    # smells like dp, could see if caching helps if this ends up being bottleneck
    def max_expected_log_prob(var_indices=tuple(range(len(random_vars))), clue_indices=tuple(range(len(clues)))):
        if not clue_indices:
            return 0.0, tuple()
        
        clue_index, *remaining_clue_indices = clue_indices
        remaining_clue_indices = tuple(remaining_clue_indices)
                
        max_expected_log_probability = -math.inf
        best_guess = None

        # try each available random variable and see which yields best match probability
        for i, var_index in enumerate(var_indices):
            remaining_var_indices = var_indices[:i] + var_indices[i + 1:]
            
            max_subproblem_expected_log_probability, guess = max_expected_log_prob(remaining_var_indices, remaining_clue_indices)
            expected_log_probability = max_subproblem_expected_log_probability + random_vars[var_index].log_expectation(keyword_to_log_prob_given_clue[clue_index])
            
            # update max log probability and corresponding code guess
            if expected_log_probability > max_expected_log_probability:
                max_expected_log_probability = expected_log_probability
                best_guess = (var_index,) + guess
                
        return max_expected_log_probability, best_guess
    
    return max_expected_log_prob()

# naive log-probability strategy (too specific, but works to show concept)
def simple_log_prob_clue_and_keyword(clue, keyword):
    return (0.0 if clue == keyword else -math.inf)
    
# test same behavior 

random_vars = [RandomVariable(random_var) for random_var in random_vars]

# try every code
codes = list(permutations(range(len(keywords)), 3))
clues = [tuple(keywords[code_num] for code_num in code) for code in codes]
print(clues[:5])


simple_log_prob_and_guess = partial(max_expected_log_probability_guess, simple_log_prob_clue_and_keyword)
log_probabilities_and_guesses = [simple_log_prob_and_guess(random_vars, clue) for clue in clues]

probabilities = [math.exp(log_probability) for log_probability, _ in log_probabilities_and_guesses]
guesses = [guess for _, guess in log_probabilities_and_guesses]

# check that probabilities made sense and that each guess was correct
print(set(probabilities)) # should all be 1
print(guesses == codes) # should be True

[('OGRE', 'FIRE', 'PERFUME'), ('OGRE', 'FIRE', 'DOG'), ('OGRE', 'PERFUME', 'FIRE'), ('OGRE', 'PERFUME', 'DOG'), ('OGRE', 'DOG', 'FIRE')]
{1.0}
True
